In [1]:
# Import dependencies

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import requests
import time
import json
from scipy.stats import linregress
from config import college_key

In [2]:
# Setup base url
base_url = "https://api.data.gov/ed/collegescorecard/v1/schools.json?"
# schools = ["University of Arizona", "Arizona State University", \
#           "University of California, Berkeley", "University of California, Los Angeles", \
#           "University of Colorado Boulder", "University of Oregon", "Oregon State University", \
#           "University of Southern California", "Stanford University", "University of Utah", \
#           "University of Washington", "Washington State University"]
schools = ["236939", "104179", "104151", "110635", "110662", "126614", \
              "209551", "209542", "123961", "243744", "230764", "236948", "236939"] 
#double checked the json return and these all seem correct^^
#use this to make a loop of which years you want. format is: range(start,end,step). 
#this gives you 2010, and 2011 data, ends before 2012.


In [3]:
# Request data as json
params = {
"id": 209551,
"_fields": ','.join(["id", "school.name", "school.state", "2017"]),
"api_key": college_key
}
data = requests.get(base_url,params=params).json()

print(json.dumps(data, indent=4))    

{
    "metadata": {
        "total": 1,
        "page": 0,
        "per_page": 20
    },
    "results": [
        {
            "2017.academics.program_percentage.computer": 0.0174,
            "2017.completion.outcome_percentage.full_time.first_time.8yr.still_enrolled": 0.0046,
            "2017.academics.program.bachelors.communications_technology": 0,
            "2017.completion.completion_rate_4yr_150_black": 0.6044,
            "2017.academics.program.degree.military": 0,
            "2017.academics.program_percentage.transportation": 0.0,
            "2017.academics.program_percentage.business_marketing": 0.1546,
            "2017.academics.program.assoc.communications_technology": 0,
            "2017.academics.program.certificate_lt_4_yr.library": 0,
            "2017.completion.completion_cohort_4yr_150_white": 2866,
            "2017.academics.program.degree_or_certificate.personal_culinary": 0,
            "school.state": "OR",
            "2017.academics.program.certificat

In [4]:
schools = ["236939", "104179", "104151", "110635", "110662", "126614", \
              "209551", "209542", "123961", "243744", "230764", "236948"] 
years = range(2002,2019,1)

years_list = []
name_of_colleges = []
college_state = []
in_state_tuition = []
out_of_state_tuiition = []
admissions_rate_overall = []
student_enrollment = []
student_size = []


for school in schools:
        params = {
            "id": school,
            "_fields": ','.join(["id", "school.name", "school.state",]), 
            "api_key": college_key
        }
        
        data = requests.get(base_url,params=params).json()
        #print(json.dumps(data, indent=4))
        for year in years:
            params = {
            "id": school ,
            "_fields": ','.join([
                                "school.name", 
                                "school.state",
                                str(year) + ".cost.tuition.out_of_state",
                                str(year) + ".cost.tuition.in_state",
                                str(year) + ".admissions.admission_rate.overall",
                                str(year) + ".student.enrollment.undergrad_12_month",
                                str(year) + ".student.size"
            ]) ,
            "api_key": college_key
            }
            
            data = requests.get(base_url,params=params).json()
            #print(json.dumps(data['results']))
            
            years_data = str(year)
            
            name_data = json.dumps(data['results'][0]['school.name'])
            college_state_data = json.dumps(data['results'][0]['school.state'])
            in_state_tuition_data = json.dumps(data['results'][0][str(year)+".cost.tuition.in_state"])
            out_of_state_tuition_data = json.dumps(data['results'][0][str(year) + ".cost.tuition.out_of_state"])
            admissions_rate_overall_data = json.dumps(data['results'][0][str(year)+".admissions.admission_rate.overall"])
            enrollment_data = json.dumps(data['results'][0][str(year)+".student.enrollment.undergrad_12_month"])
            student_size_data = json.dumps(data['results'][0][str(year)+".student.size"])
            
            years_list.append(years_data)
            name_of_colleges.append(name_data)
            college_state.append(college_state_data)
            in_state_tuition.append(in_state_tuition_data)
            out_of_state_tuiition.append(out_of_state_tuition_data)
            admissions_rate_overall.append(admissions_rate_overall_data)
            student_enrollment.append(enrollment_data)
            student_size.append(student_size_data)


college_data_df = pd.DataFrame({
    "Year": years_list,
    "College_names": name_of_colleges, 
    "College_state": college_state, 
    "In_state_tuition": in_state_tuition, 
    "Out_of_state_tuition": out_of_state_tuiition, 
    "Admissions_rate_overall": admissions_rate_overall,
    "Enrollment" : student_enrollment, 
    "Student Size": student_size
})

college_data_df     



KeyboardInterrupt: 

In [ ]:
college_data_df.head()

In [ ]:
college_data_df.to_csv('college_data.csv', encoding='utf-8', index=False)
